# Afinamiento de modelos de lenguaje para dominios específicos

## Paso 1: Fine tuning de un modelo con dataset tipo 'instruct' 

El ajuste fino con datos tipo "instruct" adapta los modelos de lenguaje (LLMs) para responder eficazmente a instrucciones específicas.

### **Características clave**:
1. **Entrenamiento con pares de instrucciones y respuestas**: Se utilizan ejemplos como "Resume este texto en dos frases" y su respuesta esperada, para enseñar al modelo a seguir comandos.
2. **Datos etiquetados o supervisados**: Las respuestas pueden ser creadas por humanos o generadas por modelos existentes y revisadas manualmente.

### **Proceso**:
1. **Creación del dataset**: Se compilan instrucciones y respuestas alineadas con los objetivos del usuario.
2. **Ajuste fino supervisado (SFT)**: El modelo se entrena utilizando estos pares para mejorar su capacidad de responder a comandos de manera precisa y alineada con la intención del usuario.

In [ ]:
!pip install bitsandbytes > /dev/null 2>&1
!pip install datasets > /dev/null 2>&1
!pip install peft > /dev/null 2>&1
!pip install wandb > /dev/null 2>&1
!pip install trl > /dev/null 2>&1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import wandb
import os

os.environ['WANDB_NOTEBOOK_NAME'] = 'nb01-instruct.ipynb'

### 1. Importar el modelo ya pre-entrenado y el tokenizador

El modelo `unsloth/mistral-7b-instruct-v0.2-bnb-4bit` es una variante compacta del modelo Mistral de 7 mil millones de parámetros, diseñada para tareas de instrucción. Utiliza técnicas de cuantización en 4 bits (bnb-4bit) para optimizar el uso de memoria y permitir su ejecución en hardware menos potente sin perder demasiada precisión. Este modelo está ajustado para interpretar y responder instrucciones en lenguaje natural, siendo útil para aplicaciones en procesamiento de lenguaje natural (NLP) que requieren respuestas eficientes y precisas en un entorno optimizado.

In [ ]:
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
tokenizer_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map='auto')

### 2. Carga de los datos de entrenamiento

In [ ]:
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/training/data/list_of_strings.jsonl', split="train")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=64)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])


wandb.init(
    project='LLM_training',
    name=model_name + '_autoregressive_instruct_fine_tuning'
)

### 3. Configuración del entrenamiento mediante LoRA y SFT

- **LoRA (Low-Rank Adaptation):**  
LoRA es una técnica utilizada para ajustar modelos grandes de lenguaje de manera eficiente. En lugar de actualizar todos los parámetros del modelo durante el entrenamiento, LoRA introduce matrices adicionales de bajo rango que capturan los cambios necesarios. Esto reduce significativamente el costo computacional y de memoria, permitiendo un ajuste fino (fine-tuning) eficiente sin necesidad de almacenar o modificar todos los parámetros originales del modelo.

- **SFT (Supervised Fine-Tuning):**  
SFT se refiere al ajuste fino de un modelo de lenguaje utilizando datos etiquetados de manera supervisada. Este proceso entrena el modelo para realizar tareas específicas mediante ejemplos claros de entrada y salida, como preguntas y respuestas, traducción, o clasificación. Es un paso clave para especializar modelos generales en tareas concretas o dominios específicos.

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']  # módulos objetivo para aplicar LoRA
)

model = get_peft_model(model, lora_config)

# Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=True,
    evaluation_strategy="no",
    eval_steps=10_000,
    report_to="wandb"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    data_collator=data_collator,
)

### 4. Entrenamiento del modelo

In [ ]:
trainer.train()

### 5. Guardar el modelo

In [ ]:
trainer.save_model("./content/drive/MyDrive/training/models/fine_tuned_model_autoregressive")
tokenizer.save_pretrained("./content/drive/MyDrive/training/models/fine_tuned_model_autoregressive")

### 6. Evaluación del modelo

In [ ]:
model_path = "./drive/MyDrive/training/models/fine_tuned_model_autoregressive/"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
import torch
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Función para generar texto
def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.01
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
numeros_a_palabras = {
    1: "uno", 2: "dos", 3: "tres", 4: "cuatro", 5: "cinco",
    6: "seis", 7: "siete", 8: "ocho", 9: "nueve"
}

texto_a_numero = {v: k for k, v in numeros_a_palabras.items()}

def generar_serie_objetivo(objetivo):
    serie = []
    suma_actual = 0
    while suma_actual < objetivo:
        numero = random.randint(1, min(9, objetivo - suma_actual))
        serie.append(numero)
        suma_actual += numero
    return serie


def comprobar_suma(cadena):
    numero_inicial = int(cadena.split()[0])

    palabras = cadena.split()[3:]  # Ignorar "es igual a"
    suma_numeros = sum(texto_a_numero.get(palabra, 0) for palabra in palabras)
    return abs(numero_inicial - suma_numeros)

def evaluar_modelo(n=10):
    total_error = 0
    for _ in range(n):
        numero_aleatorio = random.randint(10, 50)
        serie_aleatoria = generar_serie_objetivo(numero_aleatorio)
        serie_en_palabras = " más ".join(numeros_a_palabras[numero] for numero in serie_aleatoria)
        prompt = f"<s>[INST] {numero_aleatorio} es igual a [/INST]"
        
        generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)
        for text in generated_texts:
            cadena = text.split("[INST]")[1] if "[INST]" in text else text
            error = comprobar_suma(cadena)
            total_error += error
    
    return total_error / n

mae = evaluar_modelo(100)
print(f"Error absoluto promedio (MAE): {mae}")

In [ ]:
prompt = "<s>[INST] 13 es igual a [/INST]"
generated_texts = generate_text(prompt, max_length=45, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")
    cadena = f"{text}"